# Assessing the Risk/Reward Characteristics of a Trading Algorithm

## Load Data

In [1]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# Read the trading_signals.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
signals_df = pd.read_csv(
    Path("Resources/Week15-Day2-Activity3-trading_signals.csv"),
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True)

# Review the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2019-09-06,213.260,205.2584,199.72930,1,0.0,500,0.0,106630.0,22495.0,129125.0,-0.000077,0.291250
2019-09-09,214.170,205.5470,199.87850,1,0.0,500,0.0,107085.0,22495.0,129580.0,0.003524,0.295800
2019-09-10,216.700,205.9226,200.01420,1,0.0,500,0.0,108350.0,22495.0,130845.0,0.009762,0.308450
2019-09-11,223.590,206.3634,200.21150,1,0.0,500,0.0,111795.0,22495.0,134290.0,0.026329,0.342900
2019-09-12,223.085,206.7705,200.39705,1,0.0,500,0.0,111542.5,22495.0,134037.5,-0.001880,0.340375
2019-09-13,218.750,207.0573,200.50975,1,0.0,500,0.0,109375.0,22495.0,131870.0,-0.016171,0.318700
2019-09-16,219.900,207.3707,200.63715,1,0.0,500,0.0,109950.0,22495.0,132445.0,0.004360,0.324450
2019-09-17,220.700,207.7843,200.79135,1,0.0,500,0.0,110350.0,22495.0,132845.0,0.003020,0.328450
2019-09-18,222.770,208.2149,200.97605,1,0.0,500,0.0,111385.0,22495.0,133880.0,0.007791,0.338800


## Set Up the Portfolio-Level Risk/Reward Evaluation Metrics

In [3]:
# Create a list for the column name
columns = ["Backtest"]

# Create a list holding the names of the new evaluation metrics
metrics = [
    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio",
    "Sortino Ratio"]

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


## Calculate the Metrics

### Annualized Return

In [4]:
# Calculate annualized return
portfolio_evaluation_df.loc["Annualized Return"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252
)

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.066683
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Cumulative Returns

In [5]:
# Calculate cumulative return
portfolio_evaluation_df.loc["Cumulative Returns"] = signals_df["Portfolio Cumulative Returns"][-1]

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.066683
Cumulative Returns,0.32975
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Annual Volatility

In [5]:
# Calculate annual volatility
portfolio_evaluation_df.loc["Annual Volatility"] = (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.066683
Cumulative Returns,NaN
Annual Volatility,0.137751
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Sharpe Ratio

In [6]:
# Calculate Sharpe ratio
portfolio_evaluation_df.loc["Sharpe Ratio"] = (
    signals_df["Portfolio Daily Returns"].mean() * 252) / (
    signals_df["Portfolio Daily Returns"].std() * np.sqrt(252)
)

# Review the result
portfolio_evaluation_df

,Backtest
Annualized Return,0.066683
Cumulative Returns,NaN
Annual Volatility,0.137751
Sharpe Ratio,0.484084
Sortino Ratio,NaN


### Sortino Ratio

#### Calculate downside return values

In [7]:
# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[["Portfolio Daily Returns"]].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,"Downside Returns"] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df["Portfolio Daily Returns"] < 0,
                     "Downside Returns"] = sortino_ratio_df["Portfolio Daily Returns"]**2

# Review the DataFrame
sortino_ratio_df.tail()

,Portfolio Daily Returns,Downside Returns
date,,
2019-09-13,-0.016171,0.000261
2019-09-16,0.004360,0.000000
2019-09-17,0.003020,0.000000
2019-09-18,0.007791,0.000000
2019-09-19,-0.006760,0.000046


#### Calculate the Sortino ratio

In [8]:
# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df["Portfolio Daily Returns"].mean() * 252
)

# Print the result
print(f"Annualized Return: {annualized_return}")

Annualized Return: 0.0666828549129783


In [9]:
# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df["Downside Returns"].mean()) * np.sqrt(252)
)

# Print the result
print(f"Annualized Downside STD: {downside_standard_deviation}")

Annualized Downside STD: 0.09721151675996255


In [11]:
# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc["Sortino Ratio"] = sortino_ratio

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,0.066683
Cumulative Returns,NaN
Annual Volatility,0.137751
Sharpe Ratio,0.484084
Sortino Ratio,0.685956


## Set Up the Trade-Level Risk/Reward Evaluation Metrics

In [12]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        "Stock",
        "Entry Date",
        "Exit Date",
        "Shares",
        "Entry Share Price",
        "Exit Share Price",
        "Entry Portfolio Holding",
        "Exit Portfolio Holding",
        "Profit/Loss"]
)

In [14]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row["Entry/Exit"] == 1:
        entry_date = index
        entry_portfolio_holding = row["Portfolio Holdings"]
        share_size = row["Entry/Exit Position"]
        entry_share_price = row["close"]

    elif row["Entry/Exit"] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row["close"] * row["Entry/Exit Position"])
        exit_share_price = row["close"]
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                "Stock": "AAPL",
                "Entry Date": entry_date,
                "Exit Date": exit_date,
                "Shares": share_size,
                "Entry Share Price": entry_share_price,
                "Exit Share Price": exit_share_price,
                "Entry Portfolio Holding": entry_portfolio_holding,
                "Exit Portfolio Holding": exit_portfolio_holding,
                "Profit/Loss": profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2015-02-12,2015-08-03,500.0,126.46,118.44,63230.0,59220.0,-4010.0
1,AAPL,2015-12-09,2016-01-13,500.0,115.62,97.39,57810.0,48695.0,-9115.0
2,AAPL,2016-04-25,2016-06-14,500.0,105.08,97.46,52540.0,48730.0,-3810.0
3,AAPL,2016-08-22,2018-03-26,500.0,108.51,172.77,54255.0,86385.0,32130.0
4,AAPL,2018-04-19,2018-11-30,500.0,172.80,178.58,86400.0,89290.0,2890.0
5,AAPL,2019-03-28,2019-07-22,500.0,188.72,207.22,94360.0,103610.0,9250.0
6,AAPL,2015-02-12,2015-08-03,500.0,126.46,118.44,63230.0,59220.0,-4010.0
7,AAPL,2015-12-09,2016-01-13,500.0,115.62,97.39,57810.0,48695.0,-9115.0
8,AAPL,2016-04-25,2016-06-14,500.0,105.08,97.46,52540.0,48730.0,-3810.0
9,AAPL,2016-08-22,2018-03-26,500.0,108.51,172.77,54255.0,86385.0,32130.0
